<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Breast_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'breast-cancer-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5626466%2F9293524%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240929%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240929T103810Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D63f8ad14ed41cccbd38fab1cf46c2728461ae06cc338600daac22c49227ab354304f07b4570a463b8be443fa46a4e6f29e8161ceb3b641ab47bdcd80d957085b6253dd6ebedbd0507dea2a477839d0141566f9fae9ff9111a73f3274bbf0c6bec8a43760b1af6c23d2b98281f95637150fc1b4519e359dcd36326d8fa9182f016797694f302588b62fe7f141d4b47c4c43ed1d61fde5812e5491bf6afed2444176158f94227a7a2e2f924737b36183bf95dcbfd74c71c5b2eb476ce517ef57033579f309f75af81d40d6337510d3b420997ef1234276668b80eb0d876d009624f44d10eb7d68d700d154d88a9f104e29e368c3c15ec2411ac911cb1b5b71ebc1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
train_0_path='/kaggle/input/breast-cancer-detection/train/0'
train_1_path='/kaggle/input/breast-cancer-detection/test/1'
test_0_path='/kaggle/input/breast-cancer-detection/test/0'
test_1_path='/kaggle/input/breast-cancer-detection/test/1'
valid_0_path='/kaggle/input/breast-cancer-detection/valid/0'
valid_1_path='/kaggle/input/breast-cancer-detection/valid/1'


In [ ]:
train_0_files=glob.glob(train_0_path+'/*')
train_1_files=glob.glob(train_1_path+'/*')
test_0_files=glob.glob(test_0_path+'/*')
test_1_files=glob.glob(test_1_path+'/*')
valid_0_files=glob.glob(valid_0_path+'/*')
valid_1_files=glob.glob(valid_1_path+'/*')



In [ ]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]
x_valid=[]
y_valid=[]
for files in train_0_files:
    x_train.append(files)
    y_train.append(0)
for files in train_1_files:
    x_train.append(files)
    y_train.append(1)
for files in test_0_files:
    x_test.append(files)
    y_test.append(0)
for files in test_1_files:
    x_test.append(files)
    y_test.append(1)
for files in valid_0_files:
    x_valid.append(files)
    y_valid.append(0)
for files in valid_1_files:
    x_valid.append(files)
    y_valid.append(1)

In [ ]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))
print(len(x_valid))
print(len(y_valid))

In [ ]:
num_0_train=0
num_1_train=0
num_0_test=0
num_1_test=0
num_0_valid=0
num_1_valid=0
for label in y_train:
    if label==0:
        num_0_train+=1
    else:
        num_1_train+=1
for label in y_test:
    if label==0:
        num_0_test+=1
    else :
        num_1_test+=1
for label in y_valid:
    if label==0:
        num_0_valid+=1
    else :
        num_1_valid+=1
print(num_0_train)
print(num_1_train)
print(num_0_test)
print(num_1_test)
print(num_0_valid)
print(num_1_valid)

In [ ]:
import cv2
import matplotlib.pyplot as plt
image= cv2.imread(x_train[0])
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
x_train_images=[]
x_test_images=[]
x_valid_images=[]
import cv2
for i in range(len(x_train)):
    img=cv2.imread(x_train[i],cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(224,224))
    x_train_images.append(img)
for i in range(len(x_test)):
    img=cv2.imread(x_test[i],cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(224,224))
    x_test_images.append(img)
for i in range(len(x_valid)):
    img=cv2.imread(x_valid[i],cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(224,224))
    x_valid_images.append(img)

In [ ]:
import numpy as np

# Convertir les listes en tableaux NumPy
x_train_images_np = np.array(x_train_images)
x_test_images_np = np.array(x_test_images)
x_valid_images_np = np.array(x_valid_images)

# Concaténer les tableaux NumPy
data = np.concatenate((x_train_images_np, x_test_images_np, x_valid_images_np), axis=0)

# Optionnel : Afficher le shape du tableau résultant
print(f"Shape du tableau concaténé : {data.shape}")


In [ ]:
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)
y_valid_np = np.array(y_valid)


labels = np.concatenate([y_train_np, y_test_np, y_valid_np])

print(f"Shape du labels : {labels.shape}")


In [ ]:
from sklearn.utils import shuffle

# Mélanger les données d'entraînement et les étiquettes
data, labels = shuffle(data,labels)


In [ ]:

image= data[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
num_0 = np.sum(labels == 0)
num_1 = np.sum(labels == 1)
print(num_0)
print(num_1)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,  # Normalisation
                             rotation_range=15,
                             width_shift_range=0.05,
                             height_shift_range=0.05,
                             shear_range=0.1,
                             zoom_range=0.1,
                             horizontal_flip=True,
                             fill_mode='nearest')


In [ ]:
minority_class_images = [data[i] for i in range(len(labels)) if labels[i] == 1]
minority_class_images = np.array(minority_class_images)


In [ ]:
augmented_data = []
augmented_labels = []

# Calculer le nombre d'images à générer pour équilibrer les classes
num_augmented_images_needed = num_0 - num_1

for i in range(num_augmented_images_needed):
    # Sélectionner une image aléatoire parmi les images de la classe minoritaire
    img = minority_class_images[i % len(minority_class_images)]

    # Vérifier et ajuster la forme de l'image en fonction des canaux
    if len(img.shape) == 2:  # Si l'image est en 2D (224, 224), ajouter un canal (grayscale)
        img = img.reshape((1, 224, 224, 1))
    elif len(img.shape) == 3 and img.shape[-1] != 3:  # Si c'est une image couleur avec mauvaise forme
        img = img.reshape((1, 224, 224, 3))

    # Générer une nouvelle image augmentée
    for augmented_img in datagen.flow(img, batch_size=1):
        augmented_data.append(augmented_img[0])  # Ajouter l'image augmentée
        augmented_labels.append(1)  # Ajouter le label de la classe minoritaire
        break  # On génère une seule image par itération

# Convertir les listes en arrays numpy
augmented_data = np.array(augmented_data)
augmented_labels = np.array(augmented_labels)


In [ ]:
image= augmented_data[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
augmented_data = np.squeeze(augmented_data, axis=-1)  # Supprime la dernière dimension (canal)
print("Original data shape:", data[0].shape)  # (224, 224)
print("Augmented data shape:", augmented_data[0].shape)  # Cela devrait maintenant être (224, 224)


In [ ]:
image= augmented_data[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
data_balanced = np.concatenate((data, augmented_data), axis=0)
labels_balanced = np.concatenate((labels, augmented_labels), axis=0)
print("Balanced data shape:", data_balanced.shape)
print("Balanced labels shape:", labels_balanced.shape)

In [ ]:
num_0_balanced = np.sum(labels_balanced == 0)
num_1_balanced = np.sum(labels_balanced == 1)

print(f"Nombre d'images dans la classe 0 après équilibrage : {num_0_balanced}")
print(f"Nombre d'images dans la classe 1 après équilibrage : {num_1_balanced}")

In [ ]:
data_balanced, labels_balanced = shuffle(data_balanced,labels_balanced)


In [ ]:
import cv2
import matplotlib.pyplot as plt
num_images = 10
fig, axes = plt.subplots(2, 5, figsize=(15, 5))
for i, ax in enumerate(axes.flat):
    if i < num_images:

        ax.imshow(data_balanced[i])
        ax.set_title(f"Label: {labels_balanced[i]}")
        ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Vérification des formes avant la division
print(f"Data balanced shape: {data_balanced.shape}")  # (4450, 224, 224)
print(f"Labels balanced shape: {labels_balanced.shape}")  # (4450,)

# Si tout est correct, appliquez train_test_split
if data_balanced.shape[0] == labels_balanced.shape[0]:
    from sklearn.model_selection import train_test_split

    # Première division : 80% pour entraînement+validation et 20% pour test
    x_train_val, x_test, y_train_val, y_test = train_test_split(
        data_balanced, labels_balanced,
        test_size=0.2, random_state=42, stratify=labels_balanced
    )

    # Deuxième division : 80% pour entraînement et 20% pour validation (sur les données de train_val)
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val,
        test_size=0.2, random_state=42, stratify=y_train_val
    )

    # Afficher les tailles des ensembles
    print(f"Train set: {len(x_train)} samples")
    print(f"Validation set: {len(x_val)} samples")
    print(f"Test set: {len(x_test)} samples")
else:
    print("Les dimensions des données et des étiquettes sont incohérentes.")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.02,
    height_shift_range=0.02,
    zoom_range=0.01,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image= x_train[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
image= x_test[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:
image= x_val[0]
if image is not None:
    print("Image dimensions:", image.shape)
    # Afficher l'image avec Matplotlib
    plt.imshow(image)
    plt.axis('off')  # Masquer les axes
    plt.show()
else:
    print("Erreur : l'image n'a pas pu être chargée.")

In [ ]:

# Boucle pour redimensionner et s'assurer que les images ont 3 canaux
data_processed = []
for img in x_train:
    # Si l'image n'a qu'une seule dimension (grayscale)
    if len(img.shape) == 2:
        # Convertir en image RGB en dupliquant le canal
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        # Si une image a un seul canal (grayscale dans certains cas)
        img = np.repeat(img, 3, axis=-1)

    # Redimensionner à (224, 224)
    img = cv2.resize(img, (224, 224))

    # Ajouter à la liste des données traitées
    data_processed.append(img)

# Convertir la liste en array numpy (format attendu par les générateurs Keras)
x_train_processed = np.array(data_processed)

# Afficher la forme de la première image traitée
print("Shape after processing:", x_train_processed[0].shape)

# Vérifier visuellement l'image
plt.imshow(x_train_processed[0])
plt.axis('off')  # Masquer les axes
plt.show()



In [ ]:
# Boucle pour redimensionner et s'assurer que les images ont 3 canaux
data_processed = []
for img in x_test:
    # Si l'image n'a qu'une seule dimension (grayscale)
    if len(img.shape) == 2:
        # Convertir en image RGB en dupliquant le canal
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        # Si une image a un seul canal (grayscale dans certains cas)
        img = np.repeat(img, 3, axis=-1)

    # Redimensionner à (224, 224)
    img = cv2.resize(img, (224, 224))

    # Ajouter à la liste des données traitées
    data_processed.append(img)

# Convertir la liste en array numpy (format attendu par les générateurs Keras)
x_test_processed = np.array(data_processed)

# Afficher la forme de la première image traitée
print("Shape after processing:", x_test_processed[0].shape)

# Vérifier visuellement l'image
plt.imshow(x_test_processed[0])
plt.axis('off')  # Masquer les axes
plt.show()


In [ ]:
# Boucle pour redimensionner et s'assurer que les images ont 3 canaux
data_processed = []
for img in x_val:
    # Si l'image n'a qu'une seule dimension (grayscale)
    if len(img.shape) == 2:
        # Convertir en image RGB en dupliquant le canal
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        # Si une image a un seul canal (grayscale dans certains cas)
        img = np.repeat(img, 3, axis=-1)

    # Redimensionner à (224, 224)
    img = cv2.resize(img, (224, 224))

    # Ajouter à la liste des données traitées
    data_processed.append(img)

# Convertir la liste en array numpy (format attendu par les générateurs Keras)
x_val_processed = np.array(data_processed)

# Afficher la forme de la première image traitée
print("Shape after processing:", x_val_processed[0].shape)

# Vérifier visuellement l'image
plt.imshow(x_train_processed[0])
plt.axis('off')  # Masquer les axes
plt.show()


In [ ]:
train_generator = train_datagen.flow(x_train_processed, y_train, batch_size=32, shuffle=True)
test_generator = test_val_datagen.flow(x_test_processed, y_test, batch_size=32, shuffle=False)
validation_generator = test_val_datagen.flow(x_val_processed, y_val, batch_size=32, shuffle=False)



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import InceptionV3
# Charger le modèle ResNet50 pré-entraîné sans les couches supérieures
base_model =InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Gel des couches de base pour éviter la mise à jour des poids pendant l'entraînement initial
for layer in base_model.layers:
    layer.trainable = False

# Ajouter des couches personnalisées pour la classification binaire
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Réduction de la dimension
x = Flatten()(x)  # Aplatir les sorties avant de passer à la couche dense
x = Dense(256, activation='relu')(x) # Couche dense cachée
x = Dropout(0.5)(x)# Dropout pour éviter le surapprentissage
x = Dense(1, activation='sigmoid')(x)  # Couche de sortie pour la classification binaire

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=x)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

# Définir un callback pour l'early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

# Définir un callback pour sauvegarder le meilleur modèle
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
# Learning rate scheduler: réduire le taux d'apprentissage si l'amélioration stagne
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)




In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop, checkpoint,reduce_lr]
)

In [ ]:
# Débloquer les couches profondes pour le fine-tuning
for layer in base_model.layers[-20:]:  # On débloque les 20 dernières couches de base_model
    layer.trainable = True

In [ ]:

# Recompiler le modèle avec un taux d'apprentissage plus faible
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Reprendre l'entraînement avec fine-tuning
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop, checkpoint,reduce_lr]
)

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/kaggle/working/best_model.keras')

# Evaluate the model on the test set
accuracy = model.evaluate(test_generator)[1]

# Print the test accuracy
print(f"Test Accuracy: {accuracy}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).astype(int)  # Transformer les probabilités en classes 0 ou 1

# Utiliser y_test directement comme les vraies étiquettes
y_true = y_test  # Pas besoin de test_generator.classes

# ---- RAPPORT DE CLASSIFICATION ----
print("Rapport de classification :")
print(classification_report(y_true, y_pred))


In [ ]:
cm = confusion_matrix(y_true, y_pred)
print("Matrice de confusion :")
print(cm)

# ---- AFFICHAGE GRAPHIQUE DE LA MATRICE DE CONFUSION ----
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.title('Matrice de confusion')
plt.xlabel('Prédiction')
plt.ylabel('Réel')
plt.show()